In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/autism-prediction/Autism-prediction/sample_submission.csv
/kaggle/input/autism-prediction/Autism-prediction/train.csv
/kaggle/input/autism-prediction/Autism-prediction/test.csv


In [2]:
lr = 0.01
n_epochs = 20
conf = 0.80
batch_size = 35
weight = 6

In [3]:
import tensorflow as tf

from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split

In [4]:
train = pd.read_csv('../input/autism-prediction/Autism-prediction/train.csv')
test = pd.read_csv('../input/autism-prediction/Autism-prediction/test.csv')

In [5]:
pred_df = pd.DataFrame()
pred_df['ID'] = test['ID']

In [6]:
x_train = train.drop(['used_app_before', 'age_desc', 'ID', 'contry_of_res', 'ethnicity', 'Class/ASD'], axis = 1)
test = test.drop(['used_app_before', 'age_desc', 'ID', 'contry_of_res', 'ethnicity'], axis = 1)
y_train = train['Class/ASD']
x_train.head()

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,age,gender,jaundice,austim,result,relation
0,1,0,1,1,1,1,0,1,1,1,18.605397,f,no,no,7.819715,Self
1,0,0,0,0,0,0,0,0,0,1,13.829369,f,no,no,10.544296,?
2,1,1,1,1,1,1,0,0,1,1,14.679893,f,no,no,13.167506,Self
3,0,0,0,1,0,0,0,0,0,0,61.035288,f,no,no,1.530098,?
4,0,0,0,0,1,0,0,0,1,1,14.256686,m,no,yes,7.949723,Self


In [7]:
x_train.replace({'no': 0, 'yes': 1}, inplace = True)
x_train.replace({'m': 0, 'f': 1}, inplace = True)
x_train.replace({'?': 'others', 'others': 'Others'}, inplace = True)
x_train.replace({'Self':0,'?':0,'Parent':0,'Relative':0,'Others':0, 'others':0,'Health care professional':1}, inplace = True)
test.replace({'no': 0, 'yes': 1}, inplace = True)
test.replace({'m': 0, 'f': 1}, inplace = True)
test.replace({'?': 'others', 'others': 'Others'}, inplace = True)
test.replace({'Self':0,'?':0,'Parent':0,'Relative':0,'Others':0, 'others':0,'Health care professional':1}, inplace = True)

In [8]:
cols=['age', 'result']
x_train[cols] = (x_train[cols] - x_train[cols].min())/ (x_train[cols].max()- x_train[cols].min())
test[cols] = (test[cols] - test[cols].min())/ (test[cols].max()- test[cols].min())
x_train.head()

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,age,gender,jaundice,austim,result,relation
0,1,0,1,1,1,1,0,1,1,1,0.143931,1,0,0,0.651488,0
1,0,0,0,0,0,0,0,0,0,1,0.067930,1,0,0,0.821928,0
2,1,1,1,1,1,1,0,0,1,1,0.081464,1,0,0,0.986027,0
3,0,0,0,1,0,0,0,0,0,0,0.819115,1,0,0,0.258030,0
4,0,0,0,0,1,0,0,0,1,1,0.074730,0,0,1,0.659620,0


In [9]:
x_train = x_train.to_numpy()
test = test.to_numpy()
y_train = y_train.to_numpy()

In [10]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.30,  random_state=42)

In [11]:
y_train = y_train.reshape(y_train.shape[0],1)
y_val = y_val.reshape(y_val.shape[0],1)

In [12]:
initial_learning_rate =lr
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=n_epochs,
    decay_rate=0.96,
    staircase=True)

In [13]:
model=Sequential()
model.add(Dense(x_train.shape[0],input_dim=x_train.shape[1],activation='relu')) # input layer
model.add(Dense(128,activation='relu')) # hidden layers
model.add(Dense(128,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(1,activation='sigmoid')) # sigmoid output

2022-03-09 10:54:20.492187: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-09 10:54:20.648713: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-09 10:54:20.649850: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-09 10:54:20.651480: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [14]:
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.SGD(learning_rate=lr_schedule), 
              metrics=[tf.keras.metrics.BinaryAccuracy(threshold=conf),tf.keras.metrics.Recall()])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 560)               9520      
_________________________________________________________________
dense_1 (Dense)              (None, 128)               71808     
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_3 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_4 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_5 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 1

In [16]:
H = model.fit(x=x_train, y=y_train, epochs=n_epochs, batch_size=batch_size,
              validation_data=(x_val, y_val),
              class_weight = {0: 1, 1: weight})

2022-03-09 10:54:23.876276: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/20
16/16 [==============================] - 2s 29ms/step - loss: 1.4485 - binary_accuracy: 0.7750 - recall: 0.9286 - val_loss: 0.7198 - val_binary_accuracy: 0.7542 - val_recall: 1.0000
Epoch 2/20
16/16 [==============================] - 0s 7ms/step - loss: 1.4001 - binary_accuracy: 0.7750 - recall: 1.0000 - val_loss: 0.7409 - val_binary_accuracy: 0.7542 - val_recall: 1.0000
Epoch 3/20
16/16 [==============================] - 0s 7ms/step - loss: 1.3578 - binary_accuracy: 0.7750 - recall: 1.0000 - val_loss: 0.7593 - val_binary_accuracy: 0.7542 - val_recall: 1.0000
Epoch 4/20
16/16 [==============================] - 0s 7ms/step - loss: 1.3165 - binary_accuracy: 0.7750 - recall: 1.0000 - val_loss: 0.7780 - val_binary_accuracy: 0.7542 - val_recall: 1.0000
Epoch 5/20
16/16 [==============================] - 0s 7ms/step - loss: 1.2720 - binary_accuracy: 0.7750 - recall: 1.0000 - val_loss: 0.7919 - val_binary_accuracy: 0.7542 - val_recall: 1.0000
Epoch 6/20
16/16 [=====================

In [17]:
#Predicted values of y for test dataset
y_pred_test = model.predict(test)

#Creating a new dataframe with ID and predicted y values
pred_df['Class/ASD'] = y_pred_test

#Exporting the values to a csv file
pred_df.to_csv('submission.csv', index = False)